In [1]:
from typing import TypedDict, Type, Any, Callable

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from spark.config import views
from spark.create_session import create_session

from IPython.display import display

from fitter import Fitter, get_common_distributions

In [2]:
VIEWS = views("v3")
spark = create_session()

for view, file in VIEWS.items():
    df = spark.read.json(file)
    df.createOrReplaceTempView(view)


23/11/30 20:48:48 WARN Utils: Your hostname, tomek-debianbook resolves to a loopback address: 127.0.1.1; using 192.168.8.100 instead (on interface wlo1)
23/11/30 20:48:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/30 20:48:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
dfs = dict((zip(VIEWS.keys(), [spark.sql(f"SELECT * FROM {view}") for view in VIEWS.keys()])))

In [4]:
for view, df in dfs.items():
    print(view)
    display(df.toPandas())


artists


23/11/30 20:49:27 ERROR BaseAllocator: Memory was leaked by query. Memory leaked: (311808)
Allocator(toArrowBatchIterator) 0/311808/377344/9223372036854775807 (res/actual/peak/limit)

23/11/30 20:49:27 ERROR TaskContextImpl: Error in TaskCompletionListener
java.lang.IllegalStateException: Memory was leaked by query. Memory leaked: (311808)
Allocator(toArrowBatchIterator) 0/311808/377344/9223372036854775807 (res/actual/peak/limit)

	at org.apache.arrow.memory.BaseAllocator.close(BaseAllocator.java:476)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$ArrowBatchIterator.$anonfun$new$2(ArrowConverters.scala:97)
	at org.apache.spark.sql.execution.arrow.ArrowConverters$ArrowBatchIterator.$anonfun$new$2$adapted(ArrowConverters.scala:95)
	at org.apache.spark.TaskContext$$anon$1.onTaskCompletion(TaskContext.scala:132)
	at org.apache.spark.TaskContextImpl.$anonfun$invokeTaskCompletionListeners$1(TaskContextImpl.scala:144)
	at org.apache.spark.TaskContextImpl.$anonfun$invokeTaskCompletio

Py4JJavaError: An error occurred while calling o65.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:98)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:94)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:75)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:52)
	at java.base/java.lang.reflect.Method.invoke(Method.java:580)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.c

In [ ]:
class ColumnSummary(TypedDict):
    dtype: Type
    is_unique: bool
    mode: Any
    share_mode: float
    num_nulls: int
    share_nulls: float
    num_nans: int
    share_nans: float
    min: Any
    max: Any
    
def _exception_wrapper(f: Callable):
    try:
        return f()
    except Exception as e:
        return e
        
    
def get_column_summary(column: pd.Series) -> ColumnSummary:
    return ColumnSummary(
        dtype=_exception_wrapper(lambda: getattr(column, "dtype")),
        is_unique=_exception_wrapper(lambda: getattr(column, "is_unique")),
        mode=_exception_wrapper(lambda: np.concatenate(pd.DataFrame(column).mode(dropna=False).values)),
        share_mode=_exception_wrapper(lambda: column.isin(np.concatenate(pd.DataFrame(column).mode(dropna=False).values)).sum() / column.size),
        num_nulls=_exception_wrapper(column.isnull().sum),
        share_nulls=_exception_wrapper(lambda: column.isnull().sum() / column.size),
        num_nans=_exception_wrapper(column.isna().sum),
        share_nans=_exception_wrapper(lambda: column.isna().sum() / column.size),
        min=_exception_wrapper(column.min),
        max=_exception_wrapper(column.max)
    )

In [ ]:
# FOR NUMERIC DATA

TABLE = 'tracks'
COLUMN = 'duration_ms'

data = dfs[TABLE].toPandas()[COLUMN]

f = Fitter(data, distributions=get_common_distributions(), xmin=data.min(), xmax=data.max(), bins=50, density=True)
f.fit()
display(f.summary(Nbest=3))

axes = plt.gca()
axes.set_xlabel(COLUMN)
axes.set_ylabel("density")

plt.show()

In [ ]:
# FOR NON-NUMERIC DATA

TABLE = 'track_storage'
COLUMN = 'storage_class'

data = dfs[TABLE].toPandas()[COLUMN]

pd.DataFrame(data)[COLUMN].value_counts(dropna=False).plot.bar()
plt.show()


In [ ]:
summary_dfs = []

for view, df in dfs.items():
    column_names = pd.Series(dfs[view].toPandas().columns.values, name="column_name").apply(lambda name: f"{view}.{name}")
    column_summaries = pd.DataFrame.from_records(df.toPandas().apply(get_column_summary))
    s_df = pd.concat((column_names, column_summaries), axis=1)
    s_df.set_index("column_name")
    summary_dfs.append(s_df)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pd.concat(summary_dfs).set_index("column_name"))